# Task Extraction

In [1]:
import pandas as pd
import numpy as np
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
sys.path.append(src_path)
from utils.functions.task_extraction_functions import (load_file, parse_to_list, parse_llm_response,
                                                       get_meta_tasks_for_job, smart_title)
from utils.config.task_extraction_config import *
from sentence_transformers import SentenceTransformer, util
import torch
from openai import OpenAI
import pickle
import hdbscan
import ast
from sklearn.cluster import KMeans
import umap
import google.generativeai as genai
import time 
import re
from concurrent.futures import ThreadPoolExecutor
import random
from sklearn.metrics import pairwise_distances_argmin_min, silhouette_score
import plotly.express as px
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from rapidfuzz import process

In [2]:
FILE_PATHS = TASK_EXTRACTION_CONFIG['file_paths']
df_ssg = load_file(FILE_PATHS['ssg_file'])
df_umap_crosswalk = load_file(FILE_PATHS['umap_crosswalk_file'])
df_semantic_crosswalk = load_file(FILE_PATHS['semantic_crosswalk_file'])
df_esco = load_file(FILE_PATHS['esco_occupations_file'])
df_onet_tasks = load_file(FILE_PATHS['onet_tasks_file'])
df_onet_esco_crosswalk = load_file(FILE_PATHS['onet_esco_crosswalk'])
onet_descriptions = load_file(FILE_PATHS['onet_descriptions'])
dutch_descriptions = load_file(FILE_PATHS['description_nl'])

## Correct Old O*NET Codes in Task Statements

In [3]:
# Filter rows from df_onet_tasks that are not in the crosswalk
df_onet_tasks_missing = df_onet_tasks[
    ~df_onet_tasks['O*NET-SOC Code'].isin(df_onet_esco_crosswalk['O*NET Id'])
]

# View or export the result
print(f"{len(df_onet_tasks_missing)} task rows are missing from the crosswalk.")

onet_titles_missing = df_onet_tasks_missing['Title']
onet_titles_in_crosswalk = df_onet_esco_crosswalk['O*NET Title']

# Find matches based on title
matching_titles = onet_titles_missing[onet_titles_missing.isin(onet_titles_in_crosswalk)]

df_onet_tasks_with_matching_titles = df_onet_tasks_missing[
    df_onet_tasks_missing['Title'].isin(df_onet_esco_crosswalk['O*NET Title'])
]
print(f"Total rows (not just unique titles): {df_onet_tasks_with_matching_titles.shape[0]}")

title_to_code_map = df_onet_esco_crosswalk.set_index('O*NET Title')['O*NET Id'].to_dict()
# Define a function to update codes based on title match
def update_code(row):
    title = row['Title']
    if title in title_to_code_map:
        return title_to_code_map[title]
    else:
        return row['O*NET-SOC Code']  # leave unchanged

# Apply the update
df_onet_tasks['O*NET-SOC Code'] = df_onet_tasks.apply(update_code, axis=1)

# Filter rows from df_onet_tasks that are not in the crosswalk
df_onet_tasks_missing = df_onet_tasks[
    ~df_onet_tasks['O*NET-SOC Code'].isin(df_onet_esco_crosswalk['O*NET Id'])
]

# View or export the result
print(f"{len(df_onet_tasks_missing)} task rows are missing from the crosswalk.")

#Titles in crosswalk to match against
crosswalk_titles = df_onet_esco_crosswalk['O*NET Title'].unique()

# Try to match unmatched titles with similarity > 90
matches = []
for title in df_onet_tasks_missing['Title'].unique():
    match = process.extractOne(title, crosswalk_titles, score_cutoff=90)
    if match:
        matches.append((title, match[0], match[1]))

# Convert to DataFrame
df_fuzzy_matches = pd.DataFrame(matches, columns=['unmatched_title', 'matched_title', 'similarity'])

# Get corrected codes from the crosswalk
corrections = df_fuzzy_matches.merge(
    df_onet_esco_crosswalk[['O*NET Title', 'O*NET Id']],
    left_on='matched_title',
    right_on='O*NET Title',
    how='left'
)[['unmatched_title', 'matched_title', 'O*NET Id']]
# Merge the corrections into the original task DataFrame
df_onet_tasks = df_onet_tasks.merge(
    corrections,
    left_on='Title',
    right_on='unmatched_title',
    how='left'
)

# Where a match was found, replace Title and onet_soc_code
df_onet_tasks['Title'] = df_onet_tasks['matched_title'].combine_first(df_onet_tasks['Title'])
df_onet_tasks['O*NET-SOC Code'] = df_onet_tasks['O*NET Id'].combine_first(df_onet_tasks['O*NET-SOC Code'])

# Drop helper columns
df_onet_tasks = df_onet_tasks.drop(columns=['unmatched_title', 'matched_title', 'O*NET Id'])

# Final unmatched check
still_unmatched = df_onet_tasks[~df_onet_tasks['O*NET-SOC Code'].isin(df_onet_esco_crosswalk['O*NET Id'])]
print(f"Remaining unmatched rows after fuzzy title mapping: {still_unmatched.shape[0]}")

5396 task rows are missing from the crosswalk.
Total rows (not just unique titles): 1845
3551 task rows are missing from the crosswalk.
Remaining unmatched rows after fuzzy title mapping: 2560


In [4]:
unmatched_job_titles = still_unmatched['Title'].unique()

df_unmatched_titles = pd.DataFrame(unmatched_job_titles, columns=['unmatched_title'])
df_unmatched_titles.head()

,unmatched_title
0,Green Marketers
1,Methane/Landfill Gas Collection System Operators
2,Transportation Managers
3,Storage and Distribution Managers
4,Logistics Managers


## Build O*NET-SSG-ESCO crosswalk

In [5]:
FINAL_CROSSWALK_PATH = FILE_PATHS['final_crosswalk']

# Check if the file already exists
if os.path.exists(FINAL_CROSSWALK_PATH):
    print("Loading existing final_crosswalk from file...")
    final_crosswalk = pd.read_csv(FINAL_CROSSWALK_PATH)
    print(f"Loaded final_crosswalk with shape: {final_crosswalk.shape}")

else:
    print("No saved final_crosswalk found. Running full pipeline to create it...")
    
    # O*NET: Group all tasks by 'O*NET-SOC Code' to create a single descriptive text block.
    df_onet_tasks['Task'] = df_onet_tasks['Task'].astype(str)
    onet_profiles = df_onet_tasks.groupby('O*NET-SOC Code').agg({
        'Title': 'first',
        'Task': lambda tasks: ' '.join(tasks)
    }).reset_index()
    onet_profiles.rename(columns={'Task': 'onet_description'}, inplace=True)
    print(f"Created {len(onet_profiles)} unique O*NET occupation profiles.")

    # SSG: Group all 'Key Tasks' by 'Job Role' to create a single descriptive text block.
    df_ssg.dropna(subset=['Job Role', 'Key Tasks'], inplace=True)
    df_ssg['Key Tasks'] = df_ssg['Key Tasks'].astype(str)
    ssg_profiles = df_ssg.groupby('Job Role').agg({
        'Key Tasks': lambda tasks: ' '.join(tasks)
    }).reset_index()
    ssg_profiles.rename(columns={'Key Tasks': 'ssg_description'}, inplace=True)
    ssg_profiles['combined_text'] = ssg_profiles['Job Role'] + '. ' + ssg_profiles['ssg_description']
    print(f"Created {len(ssg_profiles)} unique SSG Job Role profiles.")

    model = SentenceTransformer('all-MiniLM-L6-v2')
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    print(f"Using device: {device}")

    onet_embeddings = model.encode(onet_profiles['onet_description'].tolist(), convert_to_tensor=True, show_progress_bar=True)
    ssg_embeddings = model.encode(ssg_profiles['combined_text'].tolist(), convert_to_tensor=True, show_progress_bar=True)

    cosine_scores = util.cos_sim(onet_embeddings, ssg_embeddings)
    crosswalk_data = []

    similarity_threshold = 0.7

    for i in range(len(onet_profiles)):
        best_match_index = torch.argmax(cosine_scores[i]).item()
        best_match_score = cosine_scores[i][best_match_index].item()
        
        if best_match_score >= similarity_threshold:
            best_ssg_match = ssg_profiles.iloc[best_match_index]
            onet_record = onet_profiles.iloc[i]

            crosswalk_data.append({
                'onet_soc_code': onet_record['O*NET-SOC Code'],
                'ssg_job_role': best_ssg_match['Job Role'],
                'match_similarity_score': best_match_score
            })

    df_onet_ssg_crosswalk = pd.DataFrame(crosswalk_data)
    df_onet_esco_crosswalk.rename(columns={'O*NET Id': 'onet_soc_code'}, inplace=True)
    print("O*NET-SSG crosswalk created successfully.")

    final_crosswalk = pd.merge(
        df_onet_esco_crosswalk,
        df_onet_ssg_crosswalk,
        on='onet_soc_code',
        how='left'
    )

    final_columns = [
        'onet_soc_code', 
        'O*NET Title', 
        'ssg_job_role',
        'ESCO or ISCO Title', 
        'match_similarity_score',
        'ISCO_code'
    ]
    final_crosswalk = final_crosswalk[[col for col in final_columns if col in final_crosswalk.columns]].copy()
    final_crosswalk = final_crosswalk.drop_duplicates(subset=['onet_soc_code', 'ssg_job_role', 'ISCO_code'])

    print(f"Saving final_crosswalk with shape: {final_crosswalk.shape}")
    final_crosswalk.to_csv(FINAL_CROSSWALK_PATH, index=False)
    print("Saved successfully.")


Loading existing final_crosswalk from file...
Loaded final_crosswalk with shape: (3393, 6)


In [6]:
final_crosswalk.head()

,onet_soc_code,O*NET Title,ssg_job_role,ESCO or ISCO Title,match_similarity_score,ISCO_code
0,11-1011.00,Chief Executives,NaN,secretary general,NaN,1112.7
1,11-1011.00,Chief Executives,NaN,chief executive officer,NaN,1120.3
2,11-1011.00,Chief Executives,NaN,chief operating officer,NaN,1120.4
3,11-1011.00,Chief Executives,NaN,airport chief executive,NaN,1120.3.1
4,11-1011.00,Chief Executives,NaN,social entrepreneur,NaN,1120.6


## Load and Consolidate Task Data

Need datasets that contain task descriptions or detailed occupational information. 

* O* NET: Tasks.tsv, `O*NET-SOC Code`, `Task`
* ESCO: occupations_en.csv. occupation descriptions often contain implicit task information. `conceptUri`, `preferredLabel`, `description`
* SSG: Skills Framework.csv files for different sectors. `Occupation / Job Role`. `Key Tasks`.

In [7]:
MASTER_JOBS_PATH = FILE_PATHS['crosswalk_skills_tasks']

# Check if master_jobs file already exists
if os.path.exists(MASTER_JOBS_PATH):
    print("Loading existing master_jobs from Excel...")
    master_jobs = pd.read_excel(MASTER_JOBS_PATH)
    print(f"Loaded master_jobs with shape: {master_jobs.shape}")

else:
    print("No saved master_jobs found. Running pipeline to create it...")

    # --- Merge the crosswalks to create a master job list ---
    columns_to_drop = [
        'O*NET Title', 'SSG Title', 'ESCO or ISCO Title', 'SOC Title', 
        'CIP Title', 'CIP Code', 'Army MOS Title', 'SSG Title', 'ISCO_code'
    ]
    
    master_jobs = pd.merge(
        final_crosswalk.drop(columns="match_similarity_score"),
        df_umap_crosswalk.drop(columns=[col for col in columns_to_drop if col in df_umap_crosswalk.columns]),
        on="onet_soc_code",
        how="left"
    )

    # 1. Merge O*NET Tasks
    onet_tasks_agg = df_onet_tasks.groupby('O*NET-SOC Code')['Task'].apply(list).reset_index()
    onet_tasks_agg.rename(columns={'O*NET-SOC Code': 'onet_soc_code', 'Task': 'onet_tasks'}, inplace=True)
    master_jobs = pd.merge(master_jobs, onet_tasks_agg, on='onet_soc_code', how='left')
    onet_desc_to_merge =onet_descriptions.rename(columns={'O*NET-SOC Code': 'onet_soc_code', 'Description': 'onet_description'})
    master_jobs = pd.merge(master_jobs, onet_desc_to_merge[['onet_soc_code', 'onet_description']], on='onet_soc_code', how='left')

    # 2. Merge ESCO Descriptions
    df_esco.rename(columns={'code': 'ISCO_code'}, inplace=True)
    master_jobs = pd.merge(master_jobs, df_esco[['ISCO_code', 'description']], on='ISCO_code', how='left')

    # 3. Merge SSG Tasks
    ssg_tasks_agg = df_ssg.groupby('Job Role')[['Critical Work Function', 'Key Tasks']].agg(list).reset_index()
    ssg_tasks_agg.rename(columns={'Job Role': 'ssg_job_role'}, inplace=True)
    master_jobs = pd.merge(master_jobs, ssg_tasks_agg, on='ssg_job_role', how='left')

    # Save the result
    master_jobs.to_excel(MASTER_JOBS_PATH, index=False)
    print("Saved succesfully!")

Loading existing master_jobs from Excel...
Loaded master_jobs with shape: (3393, 219)


In [8]:
master_jobs.head()

,onet_soc_code,O*NET Title,ssg_job_role,ESCO or ISCO Title,ISCO_code,O*NET Id,SOC Code,Active Learning,Active Listening,Complex Problem Solving,...,Conventional,Enterprising,Investigative,Realistic,Social,onet_tasks,onet_description,description,Critical Work Function,Key Tasks
0,11-1011.00,Chief Executives,NaN,secretary general,1112.7,11-1011.00,11-1011,1.677607,1.869583,2.557209,...,0.479637,2.570661,-0.063326,-1.725005,0.349247,"[""Direct or coordinate an organization's finan...",Determine and formulate policies and provide o...,Secretaries general head international governm...,NaN,NaN
1,11-1011.00,Chief Executives,NaN,chief executive officer,1120.3,11-1011.00,11-1011,1.677607,1.869583,2.557209,...,0.479637,2.570661,-0.063326,-1.725005,0.349247,"[""Direct or coordinate an organization's finan...",Determine and formulate policies and provide o...,Chief executive officers hold the highest rank...,NaN,NaN
2,11-1011.00,Chief Executives,NaN,chief operating officer,1120.4,11-1011.00,11-1011,1.677607,1.869583,2.557209,...,0.479637,2.570661,-0.063326,-1.725005,0.349247,"[""Direct or coordinate an organization's finan...",Determine and formulate policies and provide o...,Chief operating officers are the right hand an...,NaN,NaN
3,11-1011.00,Chief Executives,NaN,airport chief executive,1120.3.1,11-1011.00,11-1011,1.677607,1.869583,2.557209,...,0.479637,2.570661,-0.063326,-1.725005,0.349247,"[""Direct or coordinate an organization's finan...",Determine and formulate policies and provide o...,Airport chief executives lead a group of airpo...,NaN,NaN
4,11-1011.00,Chief Executives,NaN,social entrepreneur,1120.6,11-1011.00,11-1011,1.677607,1.869583,2.557209,...,0.479637,2.570661,-0.063326,-1.725005,0.349247,"[""Direct or coordinate an organization's finan...",Determine and formulate policies and provide o...,Social entrepreneurs create innovative product...,NaN,NaN


## Embeddings and Clustering

In [12]:
# Define file paths for saving
TASKS_DF_PATH = 'tasks_df.pkl'
UMAP_REDUCER_PATH = 'umap_reducer.pkl'
KMEANS_MODEL_PATH = 'kmeans_model.pkl'
REDUCED_EMB_PATH = 'reduced_embeddings_non_noise.pkl'
KMEANS_LABELS_PATH = 'kmeans_labels_non_noise.pkl'
NON_NOISE_INDICES_PATH = 'non_noise_indices.pkl'

# --- Check if saved data and models exist, otherwise run the processing ---
if all(os.path.exists(p) for p in [TASKS_DF_PATH, UMAP_REDUCER_PATH, KMEANS_MODEL_PATH, REDUCED_EMB_PATH, KMEANS_LABELS_PATH, NON_NOISE_INDICES_PATH]):
    print("Loading saved models and data...")
    tasks_df = pd.read_pickle(TASKS_DF_PATH)
    with open(UMAP_REDUCER_PATH, 'rb') as f:
        reducer = pickle.load(f)
    with open(KMEANS_MODEL_PATH, 'rb') as f:
        kmeans_umap = pickle.load(f)
    with open(REDUCED_EMB_PATH, 'rb') as f:
        reduced_embeddings_non_noise = pickle.load(f)
    with open(KMEANS_LABELS_PATH, 'rb') as f:
        kmeans_labels_non_noise = pickle.load(f)
    with open(NON_NOISE_INDICES_PATH, 'rb') as f:
        non_noise_indices = pickle.load(f)

    print("All files loaded successfully.")

else:
    print("No saved data/models found. Running embedding and clustering process...")                    

    all_tasks_flat = []

    # Columns that hold list‑like strings
    for col in ['onet_tasks', 'Key Tasks', 'Critical Work Function']:
        (
            master_jobs[col]
            .dropna()
            .apply(parse_to_list)  
            .explode()         
            .dropna()
            .apply(str.strip)    
            .pipe(all_tasks_flat.extend)  
        )

    # ESCO descriptions are already plain strings
    all_tasks_flat.extend(
        master_jobs['description']
        .dropna()
        .apply(str.strip)
        .tolist()
    )

    # De‑duplicate and sort
    unique_tasks = sorted({t for t in all_tasks_flat if t})
    tasks_df = pd.DataFrame(unique_tasks, columns=['task_text'])

    print(f"\nExtracted a total of {len(tasks_df)} unique task statements to process.")

    # --- Generate embeddings ---
    print("Generating embeddings for unique tasks...")
    model = SentenceTransformer('all-mpnet-base-v2')
    task_embeddings = model.encode(tasks_df['task_text'].tolist(), show_progress_bar=True, convert_to_tensor=False)
    print(f"Embeddings generated with shape: {task_embeddings.shape}")

    # Add embeddings to the tasks DataFrame
    tasks_df['embedding'] = list(task_embeddings)

    # Experiment with UMAP parameters before K-Means
    reducer = umap.UMAP(
        n_neighbors=15,       # A key parameter to tune
        n_components=10,      # 10-dimensional
        min_dist=0.0,         # Creates tighter clusters
        random_state=42
    )
    reduced_embeddings = reducer.fit_transform(task_embeddings)

    #Identify noisy points with HDBSCAN
    hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=15, min_samples=1, cluster_selection_epsilon=0.5, prediction_data=False) 
    hdbscan_labels = hdbscan_clusterer.fit_predict(reduced_embeddings)

    noise_indices = np.where(hdbscan_labels == -1)[0]
    non_noise_indices = np.where(hdbscan_labels != -1)[0]

    print(f"Identified {len(noise_indices)} noisy data points out of {len(tasks_df)}.")

    tasks_df_non_noise = tasks_df.iloc[non_noise_indices].copy()
    reduced_embeddings_non_noise = reduced_embeddings[non_noise_indices]

    print(f"Proceeding with K-Means on {len(tasks_df_non_noise)} non-noisy data points.")

    # Now run KMeans on the reduced_embeddings
    kmeans_umap = KMeans(n_clusters=60, random_state=42, n_init=10)
    kmeans_labels_non_noise = kmeans_umap.fit_predict(reduced_embeddings_non_noise)
    tasks_df['cluster_id'] = -1
    tasks_df.loc[non_noise_indices, 'cluster_id'] = kmeans_labels_non_noise

    # --- Save the processed data and models ---
    print("Saving data and models...")
    tasks_df.to_pickle(TASKS_DF_PATH)
    with open(UMAP_REDUCER_PATH, 'wb') as f:
        pickle.dump(reducer, f)
    with open(KMEANS_MODEL_PATH, 'wb') as f:
        pickle.dump(kmeans_umap, f)
    with open(REDUCED_EMB_PATH, 'wb') as f:
        pickle.dump(reduced_embeddings_non_noise, f)
    with open(KMEANS_LABELS_PATH, 'wb') as f:
        pickle.dump(kmeans_labels_non_noise, f)
    with open(NON_NOISE_INDICES_PATH, 'wb') as f:
        pickle.dump(non_noise_indices, f)
    print("Saved successfully.")

No saved data/models found. Running embedding and clustering process...

Extracted a total of 20831 unique task statements to process.
Generating embeddings for unique tasks...


Batches:   0%|          | 0/651 [00:00<?, ?it/s]

Embeddings generated with shape: (20831, 768)


/Users/karenherediagamero/Documents/thesis-paperwork/hjrs-thesis-karen/.venv/lib/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallel

Identified 58 noisy data points out of 20831.
Proceeding with K-Means on 20773 non-noisy data points.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saving data and models...
Saved successfully.


## OpenRouter

In [13]:
COMPARISON_LLM_PATH = FILE_PATHS['comparison_llms']

if os.path.exists(COMPARISON_LLM_PATH):
    print("Loading existing file...")
    comparison_df = pd.read_excel(COMPARISON_LLM_PATH)
    print("Loaded file.")
else:
    print("No saved files found. Running pipeline to create it...")
    OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")

    client = OpenAI(
      base_url="https://openrouter.ai/api/v1",
      api_key=OPENROUTER_API_KEY,
    )

    def query_openrouter(prompt: str, model_name: str):
        """Sends prompt to a specified model via OpenRouter."""
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f" > Error with model '{model_name}': {e}")
            return None
        
    MODELS_TO_TEST = [
        "google/gemini-2.5-flash-lite",    
        "meta-llama/llama-3-8b-instruct",  
        "qwen/qwen-2.5-72b-instruct",
        "qwen/qwen-2.5-7b-instruct",
        "microsoft/phi-4",
        "deepseek/deepseek-chat"         
    ]

    all_results = {}

    for cluster_id in range(kmeans_umap.n_clusters):
        print(f"\n--- Processing Cluster {cluster_id}/{kmeans_umap.n_clusters-1}---")

        # Prepare the prompt
        cluster_tasks_df = tasks_df[tasks_df['cluster_id'] == cluster_id]
        if cluster_tasks_df.empty:
            print(f"Cluster {cluster_id} is empty. Skipping")
            continue

        sample_size = min(20, len(cluster_tasks_df))
        sample_tasks = cluster_tasks_df.sample(n=sample_size, random_state=42)['task_text'].tolist()
        task_list_str = "\n- ".join(sample_tasks)

        prompt = f"""
            You are an expert in work analysis. Given a list of job tasks, your goal is to synthetize them into a single **standardized meta-task**.
            In other words, the Title should be a label that captures the common purpose or activity described in the tasks.

            The description of this meta-task should be phrased in an easy, understandable language for users.
            The description explains the category of activities. Avoid using "meta-task" in the description. 
            Use the most representative of the tasks as an example.
            For example: 
            Title: Supply Chain Management and Logistics
            Description: Activities related to procurement, inventory management, and flow of materials and goodsm from initial ordering and receiving to final distribution and accounting
            E.g., Administer systems and programs to reduce loss, maintain inventory control, or increase safety.
            
            Output strictly in this format:
            Meta-task: [Short phrase that summarizes that action or goal of the tasks]
            Description: [1-2 sentence explanation of what the tasks aim to accomplish or support + one example]

            Here is the list of tasks:
            - {task_list_str}
            """   
        
        cluster_results = {}
        with ThreadPoolExecutor(max_workers=len(MODELS_TO_TEST)) as executor:
            # create a future for each model call
            future_to_model = {
                executor.submit(query_openrouter, prompt, model_name): model_name
                for model_name in MODELS_TO_TEST
            }

            # process results as they complete
            for future in future_to_model:
                model_name = future_to_model[future]
                try:
                    raw_response = future.result()
                    if raw_response:
                        parsed_response= parse_llm_response(raw_response)
                        cluster_results[model_name] = parsed_response
                    else:
                        cluster_results[model_name] ={'title': 'Error: no response', 'description': ''}
                except Exception as e:
                    print(f" > Failed to parse response from '{model_name}': {e}")
                    cluster_results[model_name] = {'title': f'Error: {e}', 'description': ''}
        
        all_results[cluster_id] = cluster_results
        time.sleep(1)
    print("\n--- LLM Processing Complete ---")

    # 4. Compare the Results
    comparison_data = []
    for cluster_id, models_output in all_results.items():
        for model_name, output in models_output.items():
            comparison_data.append({
                'Cluster ID': cluster_id,
                'Model': model_name,
                'Generated Title': output.get('title', 'N/A'),
                'Generated Description': output.get('description', 'N/A')
            })

    comparison_df = pd.DataFrame(comparison_data)
    comparison_df.to_excel(COMPARISON_LLM_PATH, index=False)
    print("Saved successfully!")

Loading existing file...
Loaded file.


In [14]:
comparison_df.head()

,Cluster ID,Model,Generated Title,Generated Description
0,0,google/gemini-flash-1.5,Metering System Management and Maintenance\n\n...,This involves overseeing all aspects of meteri...
1,0,meta-llama/llama-3-8b-instruct,Meter Maintenance and Quality Control\nDescrip...,Ensures that metering equipment and sensors ar...
2,0,qwen/qwen-2.5-72b-instruct,Meter and Sensor Management\nDescription: Acti...,"Activities related to the installation, testin..."
3,0,qwen/qwen-2.5-7b-instruct,Meter and Sensor Systems Management\nDescripti...,Activities focused on ensuring the proper func...
4,0,microsoft/phi-4,Metering and Sensor Management\n\nDescription:...,This category encompasses activities related t...


## LLM-Generated Meta-tasks

In [15]:
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY") 
genai.configure(api_key=GEMINI_API_KEY)

MAX_RETRIES = 5
INITIAL_WAIT = 1

# --- Initialize the Model ---
model = genai.GenerativeModel('gemini-2.5-flash-lite')

# --- Iterate Through Clusters and Query the LLM ---
llm_meta_tasks = {} 

LLM_META_TASKS_PATH = 'llm_meta_tasks_dict.pkl'

if os.path.exists(LLM_META_TASKS_PATH):
    print("\nLoading existing LLM-generated meta-tasks...")
    with open(LLM_META_TASKS_PATH, 'rb') as f:
        llm_meta_tasks = pickle.load(f)
    print("LLM-generated meta-tasks loaded successfully.")
else:
    print("\nNo saved LLM-generated meta-tasks found. Querying LLM...")
    for cluster_id in range(kmeans_umap.n_clusters):
        print(f"Processing Cluster {cluster_id}/{kmeans_umap.n_clusters-1}...")

        # Get up to 20 sample tasks from the current cluster
        cluster_tasks_df = tasks_df[tasks_df['cluster_id'] == cluster_id]
        cluster_tasks_df = tasks_df[tasks_df['cluster_id'] == cluster_id]
        if cluster_tasks_df.empty:
            print(f"Cluster {cluster_id} is empty. Skipping.")
            continue
        sample_size = min(20, len(cluster_tasks_df))
        sample_tasks = cluster_tasks_df.sample(n=sample_size, random_state=42)['task_text'].tolist()

        # Format the tasks into a bulleted list for the prompt
        task_list_str = "\n- ".join(sample_tasks)

        # Create the prompt
        prompt = f"""
        You are an expert in work analysis. Given a list of job tasks, your goal is to synthetize them into a single **standardized meta-task**.
        In other words, the Title should be a label that captures the common purpose or activity described in the tasks.

        The description of this meta-task should be phrased in an easy, understandable language for users.
        The description explains the category of activities. Avoid using "meta-task" in the description. 
        Use the most representative of the tasks as an example.
        For example: 
        Title: Supply Chain Management and Logistics
        Description: Activities related to procurement, inventory management, and flow of materials and goodsm from initial ordering and receiving to final distribution and accounting
        E.g., Administer systems and programs to reduce loss, maintain inventory control, or increase safety.
        
        Output strictly in this format:
        Meta-task: [Short phrase that summarizes that action or goal of the tasks]
        Description: [1-2 sentence explanation of what the tasks aim to accomplish or support + one example]

        Here is the list of tasks:
        - {task_list_str}
        """
        success = False
        attempt = 0
        wait_time = INITIAL_WAIT

        while not success and attempt < MAX_RETRIES:
            try:
                # Call the API
                response = model.generate_content(prompt)
                # Extract raw text safely
                response_text = getattr(response, 'text', None)
                if not response_text:
                    raise ValueError("Empty or missing response.text")  
                print(f"Raw response:\n{response_text}\n")     
                # Parse the response to extract Title and Description
                parts = [line.strip() for line in response_text.strip().split('\n') if line.strip()]
                if len(parts) < 2 or not parts[0].startswith("Meta-task:") or not parts[1].startswith("Description:"):
                    raise ValueError(f"Unexpected format: {response_text}")
                title = parts[0].replace("Meta-task: ", "").strip()
                description = parts[1].replace("Description: ", "").strip()
                
                llm_meta_tasks[cluster_id] = {
                    'title': title,
                    'description': description
                }
                success = True
                
            except Exception as e:
                attempt += 1
                print(f"    > Error processing cluster {cluster_id}, attempt {attempt}/{MAX_RETRIES}: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(wait_time)
                    wait_time *= 2
                    wait_time += random.uniform(0,1)
                else: 
                    llm_meta_tasks[cluster_id] = {
                        'title': 'Error generating title',
                        'description': 'Error generating description'
                    }
                
        # A small delay to respect API rate limits
        time.sleep(1) 

    print("\nLLM processing complete.")
    # Save the LLM results after generating them
    print(f"Saving LLM-generated meta-tasks to {LLM_META_TASKS_PATH}...")
    with open(LLM_META_TASKS_PATH, 'wb') as f:
        pickle.dump(llm_meta_tasks, f)
    print("LLM-generated meta-tasks saved successfully.")


Loading existing LLM-generated meta-tasks...
LLM-generated meta-tasks loaded successfully.


In [16]:
LLM_META_DF_PATH = FILE_PATHS['llm_df_path']

if os.path.exists(LLM_META_DF_PATH):
    print("Loading existing llm_meta_df from Excel...")
    llm_meta_df = pd.read_excel(LLM_META_DF_PATH)
    print(f"Loaded llm_meta_df with shape: {llm_meta_df.shape}")
else: 
    print("No saved llm_meta_df found. Running pipeline to create it...")
    llm_meta_df = pd.DataFrame.from_dict(llm_meta_tasks, orient='index')
    llm_meta_df.reset_index(inplace=True)
    llm_meta_df.rename(columns={'index': 'meta_task_id'}, inplace=True)
    llm_meta_df.to_excel(LLM_META_DF_PATH, index=False)
    print("Saved succesfully!")

Loading existing llm_meta_df from Excel...
Loaded llm_meta_df with shape: (60, 3)


## Link Back

In [17]:
task_to_cluster_map = tasks_df.set_index('task_text')['cluster_id'].to_dict()
llm_meta_df.rename(columns={'meta_task_id': 'cluster_id'}, inplace=True)
id_to_title_map = llm_meta_df.set_index('cluster_id')['title'].to_dict()
id_to_description_map = llm_meta_df.set_index('cluster_id')['description'].to_dict()

print("Mapping meta-task IDs to each job. This may take a moment...")
master_jobs['meta_task_ids'] = master_jobs.apply(
    lambda row: get_meta_tasks_for_job(row, task_to_cluster_map),
    axis=1
)
print("Finished mapping meta-task IDs.")

print("Adding meta-task titles and descriptions...")
master_jobs['meta_task_titles'] = master_jobs['meta_task_ids'].apply(
    lambda ids: [id_to_title_map.get(id, 'Title Not Found') for id in ids]
)
master_jobs['meta_task_descriptions'] = master_jobs['meta_task_ids'].apply(
    lambda ids: [id_to_description_map.get(id, 'Description Not Found') for id in ids]
)

Mapping meta-task IDs to each job. This may take a moment...
Finished mapping meta-task IDs.
Adding meta-task titles and descriptions...


In [18]:
talent_landscape = load_file(FILE_PATHS['talent_landscape'])
talent_landscape.rename(columns={'isco_code':'ISCO_code_truncated'}, inplace=True)
talent_landscape.head()

,ISCO_code_truncated,PC1,PC2,PC3,education_level,title_nl,title_en,PRESET_PC2,PRESET_PC3,PRESET_OBSERVE,PRESET_THINK,PRESET_DECIDE,PRESET_ORGANIZE,PRESET_ACT,PRESET_WORKFLOW
0,1112,-10.886858,3.406035,2.583309,3.5,Ambtenaren in beleidvoerende functies,Senior government officials,74,89,72,50,54,57,47,53
1,1113,-13.862606,3.062929,1.625645,3.5,Traditionele dorpshoofden,Traditional chiefs and heads of villages,73,72,67,52,55,53,46,53
2,1114,-7.123235,2.303237,2.028675,3.5,Beleidvoerende medewerkers van belangenorganis...,Senior officials of special-interest organizat...,65,81,66,58,56,54,42,55
3,1120,-11.039758,2.823971,1.984612,3.5,Directeuren van grote ondernemingen,Managing directors and chief executives,71,80,68,54,55,52,42,54
4,1211,-12.206744,-1.440908,-0.116902,3.5,Leidinggevende functies op financieel gebied,Finance managers,36,43,41,48,46,50,51,45


In [19]:
umap_jobs = pd.read_csv(FILE_PATHS['umap_jobs'])
master_jobs_neccessary_info = master_jobs[['onet_soc_code', 'O*NET Title', 'ESCO or ISCO Title', 'description', 'ISCO_code', 'onet_description','meta_task_ids']].copy()
master_jobs_neccessary_info = master_jobs_neccessary_info.rename(columns={
    'onet_soc_code' : 'job_code'
})

dutch_descriptions.rename(columns={'code': 'ISCO_code', 'description': 'description_nl', 'preferredLabel': 'title_nl'}, inplace=True)
master_jobs_neccessary_info = master_jobs[['onet_soc_code', 'O*NET Title', 'ESCO or ISCO Title', 'description', 'ISCO_code', 'onet_description','meta_task_ids']].copy()
master_jobs_neccessary_info = pd.merge(master_jobs_neccessary_info, dutch_descriptions[['ISCO_code', 'title_nl', 'description_nl']], on='ISCO_code', how='left')

master_jobs_with_umap = pd.merge(
    master_jobs_neccessary_info,
    umap_jobs[['O*NET Title', 'UMAP_1', 'UMAP_2', 'UMAP_3']],
    on='O*NET Title',
    how='left'
)

master_jobs_with_umap = master_jobs_with_umap.rename(columns={'ESCO or ISCO Title': 'job_title'})
master_jobs_with_umap["job_title"] = master_jobs_with_umap["job_title"].apply(smart_title)
master_jobs_with_umap["title_nl"] = master_jobs_with_umap["title_nl"].apply(smart_title)
master_jobs_with_umap.dropna(inplace=True)
master_jobs_with_umap['ISCO_code_truncated'] = master_jobs_with_umap['ISCO_code'].str.split('.').str[0].astype(int)
master_jobs_with_pc = pd.merge(
    master_jobs_with_umap,
    talent_landscape[['ISCO_code_truncated', 'PC2', 'PC3', 'PRESET_PC2', 'PRESET_PC3']],
    on='ISCO_code_truncated',
    how='left'
)

print(master_jobs_with_pc.shape)
master_jobs_with_pc = master_jobs_with_pc.drop_duplicates(subset=['job_title', 'PC2', 'PC3'])
master_jobs_with_pc = master_jobs_with_pc.dropna(subset=['PC2', 'PC3'])
print(master_jobs_with_pc.shape)

master_jobs_with_pc.rename(columns={
    'title_nl': 'job_title_nl',
    'ISCO_code': 'job_code'
}, inplace=True)

(3073, 17)
(2361, 17)


In [20]:
master_jobs_with_pc.head()

,onet_soc_code,O*NET Title,job_title,description,job_code,onet_description,meta_task_ids,job_title_nl,description_nl,UMAP_1,UMAP_2,UMAP_3,ISCO_code_truncated,PC2,PC3,PRESET_PC2,PRESET_PC3
0,11-1011.00,Chief Executives,Secretary General,Secretaries general head international governm...,1112.7,Determine and formulate policies and provide o...,"[0, 4, 8, 15, 21, 22, 31, 33, 40, 41, 46]",Secretaris-generaal,Secretarissen-generaal staan aan het hoofd van...,8.865166,5.306681,7.672027,1112,3.406035,2.583309,74.0,89.0
1,11-1011.00,Chief Executives,Chief Executive Officer,Chief executive officers hold the highest rank...,1120.3,Determine and formulate policies and provide o...,"[0, 4, 8, 15, 21, 22, 31, 33, 40, 41, 46]",Algemeen Directeur,Algemeen directeuren hebben de hoogste functie...,8.865166,5.306681,7.672027,1120,2.823971,1.984612,71.0,80.0
2,11-1011.00,Chief Executives,Chief Operating Officer,Chief operating officers are the right hand an...,1120.4,Determine and formulate policies and provide o...,"[0, 4, 8, 15, 21, 22, 31, 33, 40, 41, 46]",Operationeel Directeur,Operationeel directeuren zijn de rechterhand v...,8.865166,5.306681,7.672027,1120,2.823971,1.984612,71.0,80.0
3,11-1011.00,Chief Executives,Airport Chief Executive,Airport chief executives lead a group of airpo...,1120.3.1,Determine and formulate policies and provide o...,"[0, 4, 8, 15, 21, 22, 31, 33, 35, 40, 41, 46]",Algemeen Directeur Vliegveld,Algemeen directeuren van vliegvelden geven lei...,8.865166,5.306681,7.672027,1120,2.823971,1.984612,71.0,80.0
4,11-1011.00,Chief Executives,Social Entrepreneur,Social entrepreneurs create innovative product...,1120.6,Determine and formulate policies and provide o...,"[0, 4, 8, 15, 21, 22, 28, 31, 33, 40, 41, 46]",Sociaal Ondernemer,Sociaal ondernemers creëren innovatieve produc...,8.865166,5.306681,7.672027,1120,2.823971,1.984612,71.0,80.0


In [21]:
#master_jobs_with_pc.to_excel(FILE_PATHS['master_jobs_with_umap'], index=False)

## Validation

### Intertia (Within-Cluster Sum of Squares)
Inertia is a measure of how internally coherent the clusters are in K-means clustering. Formally, it's the sum of squared distances between each point and the centroid of its assigned cluster. The goal of K-means is to minimize inertia (to group data so that points are as close as possible to their cluster centers). As the number of clusters increases, inertia always decreases. However, after a certain number of clusters, the decrease in inertia becomed marginal. That's when you start overfitting. Now we are plotting inertia vs. number of clusters to detect the elbow -> the value of k (i.e., number of clusters) beyond which adding more clusters yields diminishing returns. 


You are trading off compactness for interpretability based on domain expectations.
From a purely data-driven standpoint, 40 may give more coherent clusters.

In [22]:
task_embeddings = np.array(tasks_df['embedding'].tolist()) 
INERTIA_PLOT_PATH = 'inertia_plot.png'
if os.path.exists(INERTIA_PLOT_PATH):
    print("Inertia plot already exists. Skipping plot generation.")
else:
    print("Inertia plot not found. Generating and saving...")
    inertias = []
    K_range = range(10, 100, 5)
    for k in K_range:
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        km.fit(reduced_embeddings_non_noise)
        inertias.append(km.inertia_)

    plt.figure(figsize=(10, 6))
    plt.plot(K_range, inertias, marker='o')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')
    plt.title('Elbow Method to Determine Optimal k')
    plt.grid(True)

    # Save the plot
    plt.savefig(INERTIA_PLOT_PATH)
    plt.close()
    print(f"Inertia plot saved to: {INERTIA_PLOT_PATH}")

Inertia plot not found. Generating and saving...
Inertia plot saved to: inertia_plot.png


### Silhouette Score
This evaluates how similar an item is to its own cluster vs. other clusters.

*> 0.5 = good clustering
*0.2-0.5 = acceptable
*< 0.2 = likely overlapping or poorly separated clusters.

In [23]:
score = silhouette_score(reduced_embeddings_non_noise, kmeans_labels_non_noise)
print(f"Silhouette Score for 65 clusters: {score:.4f}")

Silhouette Score for 65 clusters: 0.3824


### Qualitative Review
Inspect top-N tasks per cluster to see if they are semantically similar. Currently only extracting the single closes task to the centroid, but we can enhance this by getting the 5 closes tasks per cluster, or using average distance to centroid to filter out outliers in clusters. 

In [24]:
for cluster_id in range(60):
    print(f"\n--- Cluster {cluster_id}: {llm_meta_tasks[cluster_id]} ---")
    cluster_tasks = tasks_df[tasks_df['cluster_id'] == cluster_id]['task_text'].head(50).tolist()
    for task in cluster_tasks:
        print(f" - {task}")


--- Cluster 0: {'title': 'Educational and Student Support Services', 'description': 'This category encompasses a wide range of activities focused on educating students, supporting their development, and managing educational environments. This includes teaching, diagnosing learning needs, and developing curriculum, as well as advising students on academic and career paths. For instance, diagnosing or treating performance-related conditions such as sports injuries falls under this umbrella.'} ---
 - Accept credit applications and verify credit references to provide check-cashing authorization or to establish house credit accounts.
 - Accept payment on accounts.
 - Accept payments, and make change or write charge slips as necessary.
 - Access computerized financial information to answer general questions as well as those related to specific accounts.
 - Accountants review and analyse financial statements, budgets, financial reports, and business plans in order to check for irregularities

In [25]:
cluster_counts = tasks_df['cluster_id'].value_counts().sort_index()
print(cluster_counts)

cluster_id
-1      58
 0     546
 1     258
 2     138
 3     593
      ... 
 55    281
 56    248
 57    357
 58    153
 59    726
Name: count, Length: 61, dtype: int64


### Visualize the Embedding Space

In [26]:
PLOTLY_PATH = "umap_3d_plot.html"

if os.path.exists(PLOTLY_PATH):
    print("Plot already exists. Skipping plot generation")
else: 
    print ("Plot not found. Creating and saving 3D UMAP plot...")

    # Map cluster_id → llm_title
    cluster_to_title = llm_meta_df.set_index('cluster_id')['title'].to_dict()

    # Create a new column in tasks_df with the human-readable label
    tasks_df['title'] = tasks_df['cluster_id'].map(cluster_to_title)

    umap_3d = umap.UMAP(n_components=3, random_state=42).fit_transform(reduced_embeddings_non_noise)

    tasks_df_non_noise = tasks_df.iloc[non_noise_indices].copy()
    tasks_df_non_noise[['umap_x', 'umap_y', 'umap_z']] = umap_3d

    fig = px.scatter_3d(
        tasks_df_non_noise,
        x='umap_x',
        y='umap_y',
        z='umap_z',
        color='title',
        title='Unified Task Universe',
        opacity=0.7
    )
    fig.show()
    fig.write_html(PLOTLY_PATH)
    print(f"Plot saved to: {PLOTLY_PATH}")

Plot not found. Creating and saving 3D UMAP plot...


/Users/karenherediagamero/Documents/thesis-paperwork/hjrs-thesis-karen/.venv/lib/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Plot saved to: umap_3d_plot.html
